In [20]:
import json
import yaml
import io
import os

topics = set()
subs = {}

def expr_to_PlusCal(str):
    return str.replace(".", "_").replace("!", "~").replace("==", "=").replace("(anonymous namespace)::","").replace("::","_").replace("True","TRUE").replace("False","FALSE").replace("||","\\/").replace("&&","/\\")

def input_port_name(topic, component):
    return f'{topic}_{component}'


In [21]:

def outVariablesTLA(jsonComp):
    vars = "variables\n\t\tmsg \in Data;\n"
    
    for state_var in jsonComp["potential_state_vars"]:
        variable = expr_to_PlusCal(state_var["qualified_name"])
        initial_value = expr_to_PlusCal(str(state_var["initial-value"]["literal"]))
        vars += f"\t\t{variable} = {initial_value};\n" 
    return vars


def topicVariables():
    variables = ''
    invariants = []
    for topic in topics:
        variables += f'\t{topic} = <<>>;\n'
        invariants.append(f'\t{topic} \in SeqOf(Data, MaxQueue)')

        if topic not in subs:
            continue
        for sub in subs[topic]:
            variables += f'\t{sub} = <<>>;\n'
            invariants.append(f'\t{sub} \in SeqOf(Data, MaxQueue)')
            
    invariants_txt =  " /\\ \n".join(invariants)
    return f"""variables
    {variables}
    define 
        TypeInvariant == \n {invariants_txt}

        Response == <>({desired_output} /= <<>>)
    end define;"""

def transitionsTLA(jsonComp):
    comp_name = jsonComp["node"]
    transitions = ''
    for rb in jsonComp["reactive_behavior"]:
        if "event" in rb:
            continue
        input_topic = rb["subscriber"]["topic"]
        topics.add(input_topic)
        if input_topic not in subs:
            subs[input_topic] = set()
        inport = input_port_name(input_topic, comp_name)
        subs[input_topic].add(inport)

        transitionID = rb["subscriber"]["callback"]
        outport = rb["publisher"]["topic"]
        topics.add(outport)
        transitions += f"""
            {transitionID}:
                if {inport} /= <<>> then
                    msg := Head({inport});
                    {inport} := Tail({inport});
                    {outport} := {outport} (+) msg;
                end if;
            """
        
    for t in jsonComp["transitions"]:
        if t["type"] == "message":
            input_topic = t["topic"]
            topics.add(input_topic)
            if input_topic not in subs:
                subs[input_topic] = set()
            subs[input_topic].add(comp_name)
            inport = input_port_name(input_topic, comp_name)
            transitionID = expr_to_PlusCal(t["callback"])
            condition = expr_to_PlusCal(t["condition"])

            outputs = ""
            for o in t["outputs"]:
                outport = o["publisher"]["topic"]
                topics.add(outport)
                outputs += f"{outport} := {outport} (+) msg;"
            
            state_changes = ""
            for sc in t["state_changes"]:
                variable  = expr_to_PlusCal(sc["variable"])
                new_value =  expr_to_PlusCal(sc["new_value"])
                state_changes += f"{variable} := {new_value};\n"

            transitions += f"""
                {transitionID}:
                    if {inport} /= <<>> then
                        msg := Head({inport});
                        {inport} := Tail({inport});
                        if {condition} then
                            {state_changes}
                            {outputs}
                        end if;
                    end if;
                """
            
        elif t["type"] == "interval":
            transitionID = expr_to_PlusCal(t["interval"])
            condition = expr_to_PlusCal(t["condition"])

            outputs = ""
            for o in t["outputs"]:
                outport = o["publisher"]["topic"]
                topics.add(outport)
                outputs += f"{outport} := {outport} (+) msg;\n"
            
            state_changes = ""
            for sc in t["state_changes"]:
                variable  = expr_to_PlusCal(sc["variable"])
                new_value =  expr_to_PlusCal(sc["new_value"])
                state_changes += f"{variable} := {new_value};\n"

            transitions += f"""
                {transitionID}:
                    if {condition} then
                        {state_changes}
                        {outputs}
                    end if;
                """

    return transitions
def systemTLA(files):

    spec = ""
    comp_names = []
    topic_names = []
    for file in files:
        comp_spec, comp_name = compTLA(file)
        spec += comp_spec
        comp_names.append(comp_name)

    for topic in topics:
        topic_spec = topicTLA(topic)
        if topic_spec:
            topic_names.append(topic.capitalize())
            spec += topic_spec
    return f'''
------------------------------- MODULE rosinfer -------------------------------
EXTENDS Sequences, Integers, TLC, FiniteSets
CONSTANTS {", ".join(comp_names)}, {", ".join(topic_names)}, Data, NULL, MaxQueue

ASSUME NULL \\notin Data


\* helper functions
''' + "SeqOf(set, n) == UNION {[1..m -> set] : m \\in 0..n} \* generates all sequences no longer than n consisting of elements in set" + f'''
seq (+) elem == Append(seq, elem)

(*--fair algorithm polling
{topicVariables()}
{spec}
end algorithm; *)
====
'''


def topicTLA(topic_name):
    inports_spec = ""
    if topic_name not in subs:
        return ''
    
    for sub in subs[topic_name]:
        inports_spec += f"{sub} := {sub} (+) msg;\n"
    return f'''
    fair process {topic_name} \in {topic_name.capitalize()}
        
        begin 
            Write: 
                if {topic_name} /= <<>> then
                    msg := Head({topic_name});
                    {topic_name} := Tail({topic_name});
                    {inports_spec}
                end if;
        end process;
    '''


def compTLA(file):

    jsonComp = json.load(open(file))
    compInstance = jsonComp["node"] 
    compType = compInstance.capitalize()

    transitions = transitionsTLA(jsonComp)
    
    return f'''
    fair process {compInstance} \in {compType}
        {outVariablesTLA(jsonComp)}
        begin 
            {transitions}
        end process;
    ''', compType

desired_output = "closest_waypoint"
files =  ["./results/driving_planner.velocity_set.json"]


print(systemTLA(files))
    


------------------------------- MODULE rosinfer -------------------------------
EXTENDS Sequences, Integers, TLC, FiniteSets
CONSTANTS Velocity_set, Localizer_pose, Odom_pose, Base_waypoints, Data, NULL, MaxQueue

ASSUME NULL \notin Data


\* helper functions
SeqOf(set, n) == UNION {[1..m -> set] : m \in 0..n} \* generates all sequences no longer than n consisting of elements in set
seq (+) elem == Append(seq, elem)

(*--fair algorithm polling
variables
    	localizer_pose = <<>>;
	velocity_set = <<>>;
	obstacle = <<>>;
	odom_pose = <<>>;
	velocity_set = <<>>;
	closest_waypoint = <<>>;
	detection_range = <<>>;
	temporal_waypoints = <<>>;
	base_waypoints = <<>>;
	velocity_set = <<>>;

    define 
        TypeInvariant == 
 	localizer_pose \in SeqOf(Data, MaxQueue) /\ 
	velocity_set \in SeqOf(Data, MaxQueue) /\ 
	obstacle \in SeqOf(Data, MaxQueue) /\ 
	odom_pose \in SeqOf(Data, MaxQueue) /\ 
	velocity_set \in SeqOf(Data, MaxQueue) /\ 
	closest_waypoint \in SeqOf(Data, MaxQueue) /\ 
	det